Parsing Python AST
==================

In order to run this notebook, be sure to use

```
pip install asttokens
```

In [1]:
import ast
from protowhat.utils_ast import AstNode

code = "x.walk(steps=10*n)"
tree = ast.parse(code)

print(ast.dump(tree))
print()
print(ast.dump(tree.body[0].value.func))

Module(body=[Expr(value=Call(func=Attribute(value=Name(id='x', ctx=Load()), attr='walk', ctx=Load()), args=[], keywords=[keyword(arg='steps', value=BinOp(left=Num(n=10), op=Mult(), right=Name(id='n', ctx=Load())))]))])

Attribute(value=Name(id='x', ctx=Load()), attr='walk', ctx=Load())


Visualizing AST Via [ast-viewer](http://ast-viewer.datacamp.com/static/index.html#/editor?code=x.walk%28steps%3D10%2an%29&start=NA&grammar=python)
--------------

![](tree_graph.png)

Enhancing Python's AST
======================

In [2]:
from example_helper import patch_ast

"""
For the sake of demonstration, this makes it so nodes in the builtin 
ast module have the following methods:

    * get_text(code): returns the code corresponding to that node.
    * _fields: returns nodes field names (e.g. module has a body field)
    * _priority: can ignore for now

See protowhat.utils_ast.AstNode for the required methods an AST class needs
"""
patch_ast()

print(tree.get_text(code))
print(tree.body[0].value.func.get_text(code))

x.walk(steps=10*n)
x.walk


Connecting Parser to Protowhat's Dispatcher
===========================================

In [3]:
from inspect import isclass
from protowhat.selectors import Dispatcher

# see protowhat.utils_ast.AstModule for a template
class AstParser:
    ParseError = SyntaxError     # what ast.parse raises for errors
    AstNode = ast.AST             # parent AST class
    classes = {v.__name__: v for v in ast.__dict__.values()
                       if isclass(v) and issubclass(v, ast.AST)}
    
    def parse(self, code, strict = True):
        # strict option is for parsers that can do partial parsing
        # of bad syntax. For example, antlr-plsql raises an error
        # when strict is set to true and the code has bad syntax.
        return ast.parse(code)

# first argument tells dispatcher the names of all node classes
# that can appear on AST, so an SCT can search for them by name
d = Dispatcher(nodes = AstParser.classes, ast = AstParser())

code = "x.walk(steps=10*n)"
tree = d.parse(code)

expr  = d('Expr', 0, tree)
expr is tree.body[0]

names = d('Name', slice(None), tree, priority = 99)
names[0].get_text(code)    # x
names[1].get_text(code)    # n

'n'

Adding Dispatcher to Protowhat State
====================================

In [4]:
from protowhat.State import State
from protowhat.Reporter import Reporter
from protowhat.checks.check_funcs import check_node, check_edge

class PythonState(State):
    def get_dispatcher(self):
        return Dispatcher(nodes = AstParser.classes, ast = AstParser())

s = PythonState(
        student_code = "print(1 + 1)",
        solution_code = "print(1 + 2); 2 + 2",
        pre_exercise_code = "",
        student_conn = None,
        solution_conn = None,
        student_result = "2",
        solution_result = "3",
        reporter = Reporter()
        )

# s.ast_dispatcher.nodes         # long print out of AST node classes
s.ast_dispatcher.ParseError
s.student_ast

expr = s.ast_dispatcher('Expr', 0, s.student_ast)
s.ast_dispatcher.ParseError
s_expr = check_node(s, 'Expr', 0)
s_call = check_node(s_expr, 'Call', 0)

s_call.student_ast    # call node
s_call.student_ast.get_text(s.student_code)

#check_node(s, 'Expr', 1)     # raises TestFail with feedback

'print(1 + 1)'

Test Yourself
--------------

1. Graph the relationship between the PythonParser, Dispatcher, and PythonState
2. Why do we go to the trouble of building the Dispatcher?
3. Build a state with the code `for ii in range(10): print(ii)`. 
   Use a `check_node` single `check_node` to get the `print` call.
   Use several `check_node` and `check_edge` calls to get the same node.